## 라이브러리 임포트

In [1]:
import re

## .txt 파일 추출

In [2]:
# 추출 경로
data_path=r'G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\2. 가공 텍스트\R078r3am1e_unprocessed.txt'

# 텍스트 데이터 추출
with open(data_path, 'rb') as source:
    lines = source.readlines()
    lines = [element.decode('utf-8') for element in lines]

text_sum = ''.join(lines)    
print(text_sum)

Paragraphs 2.31. to 2.32., renumber to read:  
"2.31.  "Braking Signal"  means a logic signal indicating when illumination of the stop lamp 
is required or allowed as specified in paragraph 5.1.17. of this Regulation.  
2.32.  "Electric Regenerative Braking System " means a braking system which, during 
deceleration, provides for th e conversion of vehicle kinetic energy into electrical 
energy and is not part of the service braking system.  
2.33.  "Disable the antilock brake system " means to put the system into a state where it will 
no longer fulfil the technical requirements in paragra ph 9 of Annex 3 to this 
Regulation. " 
Paragraph 5.1.17.2., amend to read:  
"5.1.17.2.  In addition, in case of vehicles  equipped with electric  regenerative braking systems, 
which produces a retarding force upon release of the accelerator control, the braking 
signal shall be generated also according to the following provisions 4: 
Vehicle deceleration by regenerative braking  Signal generation

## 주석 제거

In [3]:
# 주석 제거
 ## 주석 시작 행 확인
pattern_startAnnotation = r'\s[1-9]\s'
idxl_startAnnotation = [index for index, line in enumerate(lines) if re.match(pattern_startAnnotation, line)]

patten_startMain = r".*?\bto read:"
idxl_startMain = [index for index, line in enumerate(lines) if re.match(patten_startMain, line)]

pattern_endPage = r'^\[END OF PAGE\]'
idxl_endPage = [index for index, line in enumerate(lines) if re.match(pattern_endPage, line)]

idxl_endAnnotation = idxl_startMain + idxl_endPage
idxl_endAnnotation.sort()

 ## 주석 종료 행 확인 및 범위 특정
idxl_remove = []
for idx_start in idxl_startAnnotation:
    idx_cursor = idx_start
    
    while(idx_cursor not in idxl_endAnnotation):
        idxl_remove.append(idx_cursor)
        idx_cursor = idx_cursor + 1
        
 ## 주석 제거
lines_processed = lines.copy()
for idx_remove in sorted(idxl_remove, reverse=True):
    lines_processed.pop(idx_remove)

In [303]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

Paragraphs 2.31. to 2.32., renumber to read:  
"2.31.  "Braking Signal"  means a logic signal indicating when illumination of the stop lamp 
is required or allowed as specified in paragraph 5.1.17. of this Regulation.  
2.32.  "Electric Regenerative Braking System " means a braking system which, during 
deceleration, provides for th e conversion of vehicle kinetic energy into electrical 
energy and is not part of the service braking system.  
2.33.  "Disable the antilock brake system " means to put the system into a state where it will 
no longer fulfil the technical requirements in paragra ph 9 of Annex 3 to this 
Regulation. " 
Paragraph 5.1.17.2., amend to read:  
"5.1.17.2.  In addition, in case of vehicles  equipped with electric  regenerative braking systems, 
which produces a retarding force upon release of the accelerator control, the braking 
signal shall be generated also according to the following provisions 4: 
Vehicle deceleration by regenerative braking  Signal generation

In [263]:
# 테스트 - 주석 시작 행
pattern = r'\s[1-9]\s'
matched_test = [line for index, line in enumerate(lines_processed) if re.match(pattern, line)]
matched_test

[]

In [ ]:
# 테스트 - 페이지 종료 행
matched_test = [line for index, line in enumerate(lines) if "[END OF PAGE]" in line]
matched_test

## 시작 \\" 과 종료 \\" 제거

In [4]:
pattern_startWithQuote = r"^\""
idxL_startWithQuote = [index for index, line in enumerate(lines_processed) if re.match(pattern_startWithQuote, line)]

pattern_endtWithQuote = r".*\"\s\n$"
idxL_endWithQuote = [index for index, line in enumerate(lines_processed) if re.match(pattern_endtWithQuote, line)]

for idx_startWithQuote in idxL_startWithQuote:
    lines_processed[idx_startWithQuote] = lines_processed[idx_startWithQuote][1:]
    
for idx_endWithQuote in idxL_endWithQuote:
    lines_processed[idx_endWithQuote] = lines_processed[idx_endWithQuote][:-3] + lines_processed[idx_endWithQuote][-1:] 

## 특수 데이터 제거 (이미지, 테이블, 수식)

In [5]:
# 특수 데이터 제거
 ## 특수 데이터 시작 시점 확인
pattern_endWithColon = r".*\:\s{1,}\n$"
idxL_endWithColon = [index for index, line in enumerate(lines_processed) if re.match(pattern_endWithColon, line)]

patten_startMain = r".*?\bto read:"
idxl_startMain = [index for index, line in enumerate(lines_processed) if re.match(patten_startMain, line)]

pattern_endPage = r'^\[END OF PAGE\]'
idxl_endPage = [index for index, line in enumerate(lines_processed) if re.match(pattern_endPage, line)]

pattern_article = r'^\"?\s*\d{1,}\.'
idxl_aricle = [index for index, line in enumerate(lines_processed) if re.match(pattern_article, line)]

pattern_item = r"^\(.*\)"
idxl_item = [index for index, line in enumerate(lines_processed) if re.match(pattern_item, line)]

for idx_startMain in idxl_startMain: idxL_endWithColon.remove(idx_startMain)
for idx_aricle in idxl_aricle: 
    if idx_aricle in idxL_endWithColon : idxL_endWithColon.remove(idx_aricle)
idxL_cursorStart = [idx_endWithColon + 1 for idx_endWithColon in idxL_endWithColon]

idxL_cursorEnd = idxl_startMain + idxl_endPage + idxl_item
idxL_cursorEnd.sort()
 
 ## 특수 데이터 종료 시점 확인 및 데이터 범위 특정
idxL_remove = []
for idx_cursorStart in idxL_cursorStart:
    
     ### 시작 지점부터 한 줄씩 종료 조건을 확인하며 데이터 범위 탐색
    idx_cursor = idx_cursorStart
    #print(f"Start with the line IDX:{idx_moment}")
    
    while idx_cursor not in idxL_cursorEnd:
        idxL_remove.append(idx_cursor)
        #print(f" Mark the line IDX to be removed:{idx_moment}")
        
        idx_cursor = idx_cursor + 1
        
     ### 포함 여부 확인 후 작업 스케줄 수정 (특수 데이터의 범위로 특정된 부분 ~ 기존에 확인한 탐색 시작 지점)
    idxL_redundant = [ idxL_cursorStart.index(elm_redundant) for elm_redundant in list(filter(lambda x : (idx_cursorStart < x <= idx_cursor), idxL_cursorStart)) ]
    for idx_redundant in sorted(idxL_redundant, reverse=True):
        idxL_cursorStart.pop(idx_redundant)
         #print(f"Pop out the start IDX of the redundant: {idxL_start.pop(idx_redundant)}")

 ## 특정한 범위를 바탕으로 특수 데이터 제거        
for idx_remove in sorted(idxL_remove, reverse=True):
    lines_processed.pop(idx_remove)

In [317]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

Paragraphs 2.31. to 2.32., renumber to read:  
2.31.  "Braking Signal"  means a logic signal indicating when illumination of the stop lamp 
is required or allowed as specified in paragraph 5.1.17. of this Regulation.  
2.32.  "Electric Regenerative Braking System " means a braking system which, during 
deceleration, provides for th e conversion of vehicle kinetic energy into electrical 
energy and is not part of the service braking system.  
2.33.  "Disable the antilock brake system " means to put the system into a state where it will 
no longer fulfil the technical requirements in paragra ph 9 of Annex 3 to this 
Regulation. 
Paragraph 5.1.17.2., amend to read:  
5.1.17.2.  In addition, in case of vehicles  equipped with electric  regenerative braking systems, 
which produces a retarding force upon release of the accelerator control, the braking 
signal shall be generated also according to the following provisions 4: 
Insert new paragraph 5.1.17.3. , to read:  
5.1.17.3.  Once generat

## 띄어쓰기 제거 (페이지 전환 부분)

In [6]:
# 띄어쓰기 제거 (페이지 전환 부분)

 ## 페턴 
pattern_withSpace = r'^\s+'
pattern_endPage = r'^\[END OF PAGE\]'

 ## 페이지 종료 시점 확인
idxL_pageEnd = [index for index, line in enumerate(lines_processed) if re.match(pattern_endPage, line)]
idxL_pageEnd.pop()

 ## 페이지 종료 후 띄어쓰기 삽입 여부 확인 및 수정
for idx_pageEnd in idxL_pageEnd:
    if re.match(pattern_withSpace, lines_processed[idx_pageEnd+1]):
        lines_processed[idx_pageEnd+1] = lines_processed[idx_pageEnd+1][1:]

In [308]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

Paragraphs 2.31. to 2.32., renumber to read:  
2.31.  "Braking Signal"  means a logic signal indicating when illumination of the stop lamp 
is required or allowed as specified in paragraph 5.1.17. of this Regulation.  
2.32.  "Electric Regenerative Braking System " means a braking system which, during 
deceleration, provides for th e conversion of vehicle kinetic energy into electrical 
energy and is not part of the service braking system.  
2.33.  "Disable the antilock brake system " means to put the system into a state where it will 
no longer fulfil the technical requirements in paragra ph 9 of Annex 3 to this 
Regulation. 
Paragraph 5.1.17.2., amend to read:  
5.1.17.2.  In addition, in case of vehicles  equipped with electric  regenerative braking systems, 
which produces a retarding force upon release of the accelerator control, the braking 
signal shall be generated also according to the following provisions 4: 
Insert new paragraph 5.1.17.3. , to read:  
5.1.17.3.  Once generat

## 띄어쓰기 제거 (예외 처리) (개정안 ver 미사용)

In [278]:
# 띄어쓰기 제거 (예외 처리)
 
pattern_withSpace_exceptions = r'^\s\d{1,}\.\d{1,}\.'
idxL_withSpace_exceptions = [index for index, line in enumerate(lines_processed) if re.match(pattern_withSpace_exceptions, line)]

for idx_withSpace_exceptions in idxL_withSpace_exceptions:
    lines_processed[idx_withSpace_exceptions] = lines_processed[idx_withSpace_exceptions][1:]

## 단독 개행('\r\n') 및 페이지 전환('[END OF PAGE]') 제거

In [7]:
# 단독 개행 및 페이지 전환 제거

## 패턴
pattern_onlyEnter = r'^\s*\n$'
pattern_endPage = r'^\[END OF PAGE\]'

## 패턴과 일치하는 행 확인
idxL_onlyEnter_endPage = [index for index, line in enumerate(lines_processed) if (re.match(pattern_onlyEnter, line) or re.match(pattern_endPage, line))]

## 패턴과 일치하는 행 제거
for idx_remove in sorted(idxL_onlyEnter_endPage, reverse=True):
    lines_processed.pop(idx_remove)

In [310]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

Paragraphs 2.31. to 2.32., renumber to read:  
2.31.  "Braking Signal"  means a logic signal indicating when illumination of the stop lamp 
is required or allowed as specified in paragraph 5.1.17. of this Regulation.  
2.32.  "Electric Regenerative Braking System " means a braking system which, during 
deceleration, provides for th e conversion of vehicle kinetic energy into electrical 
energy and is not part of the service braking system.  
2.33.  "Disable the antilock brake system " means to put the system into a state where it will 
no longer fulfil the technical requirements in paragra ph 9 of Annex 3 to this 
Regulation. 
Paragraph 5.1.17.2., amend to read:  
5.1.17.2.  In addition, in case of vehicles  equipped with electric  regenerative braking systems, 
which produces a retarding force upon release of the accelerator control, the braking 
signal shall be generated also according to the following provisions 4: 
Insert new paragraph 5.1.17.3. , to read:  
5.1.17.3.  Once generat

## 챕터(chapter) 제목 행 병합 (개정안 ver 미사용)

In [283]:
# 챕터 제목 행 병합
pattern_chapterTitle = r'^\s{2}[a-zA-Z]'
idxL_chapterTitle = [index for index, line in enumerate(lines_processed) if (re.match(pattern_chapterTitle, line))]

for idx_chapterTitle in sorted(idxL_chapterTitle, reverse=True):
    lines_processed[idx_chapterTitle-1] = lines_processed[idx_chapterTitle-1][:-4]

## 조문 단위 행 병합

In [8]:
# 조문 행 병합

patten_startMain = r".*?\bto read:"
idxl_startMain = [index for index, line in enumerate(lines_processed) if re.match(patten_startMain, line)]

pattern_normalText = r'^[a-zA-Z]'
idxL_normalText = [index for index, line in enumerate(lines_processed) if (re.match(pattern_normalText, line))]

for idx_startMain in idxl_startMain: 
    if idx_startMain in idxL_normalText : idxL_normalText.remove(idx_startMain)

for idx_normalText in sorted(idxL_normalText, reverse=True):
    lines_processed[idx_normalText-1] = lines_processed[idx_normalText-1][:-2]

In [312]:
# 조문 행 병합 (예외 처리) (개정안 ver 미사용)
pattern_normalText_exceptions = r'^\(.{4,}\)'
idxL_normalText_exceptions = [index for index, line in enumerate(lines_processed) if (re.match(pattern_normalText_exceptions, line))]

for idx_normalText_exceptions in sorted(idxL_normalText_exceptions, reverse=True):
    lines_processed[idx_normalText_exceptions-1] = lines_processed[idx_normalText_exceptions-1][:-2]

In [9]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

Paragraphs 2.31. to 2.32., renumber to read:  
2.31.  "Braking Signal"  means a logic signal indicating when illumination of the stop lampis required or allowed as specified in paragraph 5.1.17. of this Regulation.  
2.32.  "Electric Regenerative Braking System " means a braking system which, duringdeceleration, provides for th e conversion of vehicle kinetic energy into electricalenergy and is not part of the service braking system.  
2.33.  "Disable the antilock brake system " means to put the system into a state where it willno longer fulfil the technical requirements in paragra ph 9 of Annex 3 to thisRegulation. 
Paragraph 5.1.17.2., amend to read:  
5.1.17.2.  In addition, in case of vehicles  equipped with electric  regenerative braking systems,which produces a retarding force upon release of the accelerator control, the brakingsignal shall be generated also according to the following provisions 4: 
Insert new paragraph 5.1.17.3. , to read:  
5.1.17.3.  Once generated, the signal

## .txt 파일 저장

In [322]:
# 결과 저장 경로
data_path=r'G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\2. 가공 텍스트\R078r3am1e_processed.txt'

# txt 파일 저장
with open(data_path, 'w', encoding='utf-8') as result:
    text_sum = ''.join(lines_processed)    
    result.write(text_sum)